In [ ]:
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u
from astroquery.ipac.ned import Ned
import numpy as np
from astropy.cosmology import WMAP9 as cosmo
from urllib.request import urlretrieve
from collections import Counter
import matplotlib.pyplot as plt
import pickle
from astroquery.esa.hubble import ESAHubble
from astropy.io import fits
from astropy.table import Table
import glob
import os



def read_mul171():
    # read mullaney and add designations
    mul171_ = pd.read_pickle("/home/insepien/research-data/alpaka/mull171.pkl")
    designations = []
    for i in range(0, len(mul171_)):
        pos = SkyCoord(ra=mul171_['RA'][i]*u.deg, dec=mul171_['DEC'][i]*u.deg)
        posstring = pos.to_string('hmsdms').split(' ')
        des_ra = posstring[0][0:2]+posstring[0][3:5]
        des_dec = posstring[1][0:3]+posstring[1][4:6]
        des = 'J'+des_ra+des_dec
        designations.append(des)
    mul171_['DESIG'] = designations
    return mul171_

def pos(row):
    """make skyCoord object for HST coord cone search"""
    return SkyCoord(ra=row['RA']*u.deg, dec=row['DEC']*u.deg)

def cross_bigmac(mul171):
    """crossmatch sample with big MAC"""
    # read in big  mac
    bigmac = pd.read_csv("/home/insepien/research-data/GFG.csv")
    # format designation
    desigs = []
    for i in range(len(bigmac)):
        name = bigmac['Name1'].loc[i].replace("SDSS","")
        if name[0] == "J":
            if "+" in name:
                desig = name.split("+")[0][:5] + "+" + name.split("+")[1][:4]
                desigs.append(desig)
            elif "-" in name:
                desig = name.split("-")[0][:5] + "-" + name.split("-")[1][:4]
                desigs.append(desig)
            else: print(name) 
        else:
            desigs.append(name)
    bigmac['DESIG'] = desigs

    # merge big mac and mullaney
    mul_bm = pd.merge(mul171,bigmac, on="DESIG")

    # optionally can get decals images
    # for n in mul_bm.index:
    #     urlretrieve('http://legacysurvey.org/viewer/jpeg-cutout?ra='+str(mul_bm.loc[n,'RA'])+'&dec='+str(mul_bm.loc[n,'DEC'])+'&layer=decals-dr7&pixscale=0.27&bands=grz',
    #                 "/home/insepien/research-data/hst/mul_bm/"+str(mul_bm.loc[n,'DESIG'])+'.jpg')
    return mul_bm, bigmac

def cal_sep(theta, z):
    """return dual sep in kpc given scalar angle sep in arcsec"""
    angle = (theta*u.arcsec).to(u.rad).value
    return (cosmo.angular_diameter_distance(z)*angle).to(u.kpc)


def f(on, theta):
    fn = "/home/insepien/research-data/hst/mul_bm/"+on+".jpg"
    decals_plate_scale = 0.236 #''/pix
    pix_sep = theta/decals_plate_scale

    fig,ax = plt.subplots()
    im = plt.imread(fn)
    midF = im.shape[0]/2
    ax.imshow(im)
    circ = plt.Circle((midF,midF), pix_sep, fill=False, color='white',alpha=0.5,label=f"{theta:.2f}''")
    ax.add_patch(circ)
    ax.legend()
    ax.set_title(on)


In [ ]:
res = 0.04*2.5
cal_sep(res,0.18)

### find megellan seeing

In [ ]:
fn = os.listdir('/home/insepien/research-data/agn-result/box/final_cut')
onames = [f[:10] for f in fn]

fwhms = []
min_seps = []
for on in onames:
    expfile = glob.glob(os.path.expanduser("~/raw-data-agn/exp-fits-agn/*"+on+"*.exp.fits"))[0]
    with fits.open(os.path.expanduser(expfile)) as hdul:
        hdu0 = hdul[0]
    z = mul171[mul171['DESIG']==on]['Z'].values[0]
    fwhm = hdu0.header['FWHM_AVE']
    fwhms.append(fwhm)
    min_seps.append(cal_sep(fwhm,z).value)

fig,ax = plt.subplots(1,2,figsize=(7,3))
ax[0].hist(fwhms)
ax[0].set_xlabel("Avg FWHM ['']")
ax[0].set_title(f"Min: {np.min(fwhms):.2f}, \n median: {np.median(fwhms):.2f}")

ax[1].hist(min_seps)
ax[1].set_xlabel("min resolution (kpc)")
ax[1].set_title(f"Min: {np.min(min_seps):.2f}, \n median: {np.median(min_seps):.2f}")
;

## cross-match catalogs

In [ ]:
# read in original sample
mul171 = read_mul171()
mul_bm, bigmac = cross_bigmac(mul171)

# get some stats on sub-kpc pairs for science justification
subkpc_mask = (bigmac['Primary System Type']=='Dual AGN Candidate') & (bigmac['Sep(kpc)']<1) & (bigmac['Sep(kpc)']>0)
subkpc_dual = bigmac[subkpc_mask]
dual_mask = (bigmac['Primary System Type']=='Dual AGN Candidate') & (bigmac['Sep(kpc)']>0)
dual = bigmac[dual_mask]
print("fraction of sub-kpc", len(subkpc_dual)/len(dual))
print("number of duals", len(dual))

# get some methods of measuring sub-kpc sep
anyl_meth = Counter(subkpc_dual['Parsed Analysis Method'])
print("1st most common method: ",anyl_meth.most_common(2)[0])
print("2nd most common method: ",anyl_meth.most_common(2)[1])


# cross-match specifically for magellan
with open("/home/insepien/research-data/alpaka/alpaka_39fits.pkl","rb") as f:
    magel = pickle.load(f)

magel.reset_index(inplace=True)
magel.rename(columns={"Desig":"DESIG"},inplace=True)
magel_bm, _ = cross_bigmac(magel)

In [ ]:
from astropy.io import ascii
ipac_cat = Table([mul171['RA'], mul171['DEC']], names=['ra','dec'])
for_wise_search = ascii.write(ipac_cat, 'for_wise_search.dat', format='ipac', overwrite=True);

## sample stats

In [ ]:
def print_shifted_lines(wl_rest, zmin=float(min(mul171['Z'])),zmax=float(max(mul171['Z']))):
    print(f" {wl_rest} shifted to min z:{np.floor((1+zmin)*wl_rest):.0f}, max z:{np.ceil((1+zmax)*wl_rest):.0f}")

print_shifted_lines(5007)

In [ ]:
fig,ax = plt.subplots()
ax.hist((1+mul171['Z'])*4959,histtype='step',label="shifted [OIII]4959",alpha=0.5)
ax.hist((1+mul171['Z'])*5007,histtype='step',label="shifted [OIII]5007")
ax.hist((1+mul171['Z'])*6500,histtype='step',label="shifted HA 6500")
ax.hist((1+mul171['Z'])*6548,histtype='step',label="shifted NII 6548",alpha=0.5)
ax.hist((1+mul171['Z'])*6584,histtype='step',label="shifted NII 6584",alpha=0.5)
ax.set_xlabel("wavelength (angstroms)")
ax.set_ylabel("# of sources")
ax.set_xlim((5500,8300))
ax.set_xticks(np.arange(5500,8500,300))
ax.set_title("shifted emission lines in full 171 sample")
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
;

In [ ]:
flt = {'621M':[6218.9,609.5],
       '689M':[6876.8,684.2],
       '625W':[6242.6,1464.6],
       '763M':[7614.4,708.6],
       '775W':[7651.4,1179.1]}

def onBand_mask(wl_line,flt_name):
    """mask for on band for some filter with some central wl and width"""
    wl_eff,width = flt[flt_name]
    wl_min = wl_eff - width/2
    wl_max = wl_eff + width/2
    return ((1+mul171['Z'])*wl_line>wl_min) & ((1+mul171['Z'])*wl_line < wl_max)

def offBand_mask(wl_line,flt_name):
    """mask for on band for some filter with some central wl and width"""
    wl_eff,width = flt[flt_name]
    wl_min = wl_eff - width/2
    wl_max = wl_eff + width/2
    return ((1+mul171['Z'])*wl_line<wl_min) | ((1+mul171['Z'])*wl_line > wl_max)

def test_filter_set(fb_on,fb_off,line='OIII'):
    if line=='OIII':
        mask_on_band = onBand_mask(4959,fb_on) & onBand_mask(5007,fb_on)
        mask_off_band = offBand_mask(4959,fb_off) & offBand_mask(5007,fb_off)
        print(f"Number of galaxies with both [OIII] on F{fb_on} and off F{fb_off}: {np.sum(mask_on_band&mask_off_band)}")
    elif line =="NII":
        mask_on_band = onBand_mask(6548,fb_on) & onBand_mask(6584,fb_on)
        mask_off_band = offBand_mask(6548,fb_off) & offBand_mask(6584,fb_off)
        print(f"Number of galaxies with both NII on F{fb_on} and off F{fb_off}: {np.sum(mask_on_band&mask_off_band)}")

test_filter_set("621M","689M","OIII")
test_filter_set("625W","689M","OIII")
test_filter_set("763M","689M","NII")
test_filter_set("775W","689M","NII")

In [ ]:
mul171_rep_inf = mul171.copy().replace([np.inf, -np.inf,0], 1e-99)

fig,ax = plt.subplots(1,3,figsize=(10,3))
x="FLUX"
ax[0].hist(np.log10(mul171['OIII_5007_'+x]),label='[OIII]5007')
ax[0].hist(np.log10(mul171['OIII_4959_'+x]),label='[OIII]4959')
ax[0].set_title("OIII "+x)

ax[1].hist(np.log10(mul171_rep_inf['HA_'+x]),label='HA',histtype='step')
ax[1].hist(np.log10(mul171_rep_inf['NII_6548_'+x]),label='NII 6548',histtype='step')
ax[1].hist(np.log10(mul171_rep_inf['NII_6584_'+x]),label='NII 6584')
ax[1].set_title("HA and NII "+x)

cut={'LUM':30, 'FLUX':-5}
unit={'LUM':'erg/s', 'FLUX':' 1e-17 erg/s/cm-2'}

ax[2].hist(np.log10(mul171['HA_'+x][np.log10(mul171['HA_'+x])>cut[x]]),label='HA',histtype='step')
ax[2].hist(np.log10(mul171['NII_6548_'+x][np.log10(mul171['NII_6548_'+x])>cut[x]]),label='NII 6548')
ax[2].hist(np.log10(mul171['NII_6584_'+x][np.log10(mul171['NII_6584_'+x])>cut[x]]),label='NII 6584')
ax[2].set_title("HA and NII "+x+"\n without outliers")

[a.set_xlabel(f"Log({x}) [{unit[x]}]") for a in ax]
ax[0].set_ylabel("# objs")
[a.legend() for a in ax]
fig.tight_layout();

## binary calculations

In [ ]:
from scipy.integrate import quad

def f(x,alpha,beta,phi):
    """integrand of SChecter function"""
    return phi*x**alpha*np.exp(-x**beta)  

def n_massive_dens(logMcut_,logMbhcut_):
    """calculate the number density of binary given mass cuts"""
    # GSMF constants
    phi1 = 10**-4.85
    phi2 = 10**-2.85
    Ms = 10**11.33
    alpha1 = 0.92
    alpha2 = -1.38
    # BHMF constants
    alpha_bh = -1.27
    beta_bh = 0.45
    phi_bh = 10**-2
    Msbh = 10**8.09
    # numerically integrate the number density of massive galaxies (note the GSMF is a sum of 2 PS functions)
    result1, _ = quad(f, 10**logMcut_/Ms, np.inf, args=(alpha1,1,phi1))
    result2, _ = quad(f, 10**logMcut_/Ms, np.inf, args=(alpha2,1,phi2)) 
    n_massive_gal = result1+result2
    # number density of massive BHs
    n_massive_bh, _ = quad(f, 10**logMbhcut_/Msbh, np.inf, args=(alpha_bh,beta_bh,phi_bh)) 
    return n_massive_gal, n_massive_bh

def n_bi(n_massive_,n_binary_):
    """calculate number of binary and galaxies given their number densities
        find dual fraction and expected observations"""
    # get binary number 
    # SDSS covers only part of sky
    f_sky = 9380/41253
    # comoving volume in z-range
    volume = (cosmo.comoving_volume(0.22)-cosmo.comoving_volume(0.14)).value
    # Number of binary
    N_c22 = n_binary_*volume*f_sky
    # get massive gal number
    N_massive_gal = n_massive_*volume*f_sky
    # dual fraction
    f_dual = N_c22/N_massive_gal
    # results
    print(f"{N_c22:.0f} binaries in SDSS in z=0.14-0.22")
    print(f"{N_massive_gal:.2e} massive galaxies")
    print(f"dual fraction:{f_dual*100:.2f}%")
    print(f"We can observe {f_dual*50:.0f}/50 binaries")

In [ ]:
# define cuts and binary density
logMcut = 11
logMbhcut = 8
n_bin = 3e-5
# calculate number of massive things
n_massive_gal, n_massive_bh = n_massive_dens(logMcut_=logMcut, logMbhcut_=logMbhcut)

# print results
print("GSMF")
n_bi(n_massive_gal,n_bin)
print("\nBHMF")
n_bi(n_massive_bh,n_bin)

## r-band flux distribution

In [ ]:
with fits.open("/home/insepien/research-data/alpaka/specphot_fibermag.fits",memmap=True) as hdul:
    data_specphot = hdul[1].data
    print(hdul[1].columns)

with fits.open("/home/insepien/research-data/alpaka/specobj_dec.fits",memmap=True) as hdul:
    data_specobj = hdul[1].data
    print(hdul[1].columns)

In [ ]:
mask_phot = np.isin(data_specphot['dec'],mul171['DEC'])
mask_obj = np.isin(data_specobj['dec'],mul171['DEC'])

In [ ]:
magab = 22.5 - 2.5 * np.log10(data_specobj['spectroFlux_r'][mask_obj])
plt.hist(magab)
plt.xlabel("AB mag")
plt.title(f"SpecObj for {np.sum(mask_obj):.0f} objects")

In [ ]:
plt.hist(data_specphot['fiberMag_r'][mask_phot])
plt.xlabel("AB mag")
plt.title(f"SpecPhot for {np.sum(mask_phot):.0f} objects")

## j1010 hst search

In [ ]:
esahubble = ESAHubble()
# get j1010 data 
j1010 = esahubble.query_criteria(proposal=14730)
j1010 = j1010.to_pandas()

In [ ]:
j1010[(j1010['filter']=="F621M") & (j1010['target_name']=="2MASX-J10102294+1413009")]['observation_id']

# filter and HST search

In [ ]:
def search_hst(mul171_,filters_=["F621M","F689M"]):
    """search if sample has been observed with HST
        returns the search results, indices with search errors, and indices of objects that have been observed before"""
    res = []
    erred_ind = []
    for i in range(140,len(mul171_)):
        try:
            res.append(esahubble.cone_search_criteria(coordinates=pos(mul171_.loc[i]),
                                                radius=7*u.arcmin,
                                                instrument_name = ['WFC3'],
                                                filters = filters_))
        except:
            erred_ind.append(i)
    observed_ind = [i for i in range(len(res)) if len(res[i])!=0]
    return res, erred_ind, observed_ind

In [ ]:
# see what files is in an observation
esahubble.get_associated_files(observation_id="hst_12521_0e_wfc3_uvis_total")
# download a file
esahubble.download_file(file="hst_12521_0e_wfc3_uvis_total_drz.fits")